In [1]:
import api_nyt as nyt
import api_wikipedia as wiki
import manage_articles as mng
import statistics
import datetime
import pickle
import timeseries
import TextPreprocessing as txt
from datetime import date
import graphics
import matplotlib.pyplot as plt
import matching
import time
import word
import random
import numpy

In [2]:
#Load all articles from 2019 and a shuffled copy
articles = mng.load_articles("/Users/markus/Downloads/wikinewsPickle/nyt2019.json")
articles_shuffled = mng.shuffle_publicationdates(articles)

#Get a dict of dicts for each calendar week with word frequencies from getWordCounts
wordCounts = mng.getWordCounts(articles)
wordCounts_shuffled = mng.getWordCounts(articles_shuffled)

#List of distinct words are the same for shuffled and non-shuffled articles
distinctWords = mng.getDistinctWords(wordCounts)

#List of lists of tuples containing weekly word frequency
countsPerWeek = []
for w in distinctWords:
    countsPerWeek.append((w,mng.getCountPerWeek(wordCounts,w)))

countsPerWeek_shuffled = []
for w in distinctWords:
    countsPerWeek_shuffled.append((w,mng.getCountPerWeek(wordCounts_shuffled,w)))    

#Create list of word objects for each keyword
words = []
for c in countsPerWeek:
    words.append(word.Word(c[0],ts_articles=timeseries.Timeseries(c[1])))
words = sorted(words, key=lambda x: sum(x.ts_articles.getCounts()), reverse=True)

words_shuffled = []
for c in countsPerWeek_shuffled:
    words_shuffled.append(word.Word(c[0],ts_articles=timeseries.Timeseries(c[1])))
words_shuffled = sorted(words_shuffled, key=lambda x: sum(x.ts_articles.getCounts()), reverse=True)

'\nwords = []\nwith open("/Users/markus/Downloads/wikinewsPickle/words.txt", "rb") as fp:\n    words = pickle.load(fp)\n\nwords_shuffled = []\nwith open("/Users/markus/Downloads/wikinewsPickle/words_shuffled.txt", "rb") as fp:\n    words_shuffled = pickle.load(fp)\n'

In [6]:
top_amount = 150

#Get top_amount of keywords from words, same for shuffled and non-shuffled
keywords = []
for i in range(top_amount):
    keywords.append(words[i].keyword)

#Get matching with wikipedia site
m = matching.groupmatch(keywords, articles)
m_shuffled = matching.groupmatch(keywords,articles_shuffled)

#For each returned entry from groupmatch, add wikipedia site and wiki timeseries to word object
for key in m.keys():
    #Following line from 
    #https://stackoverflow.com/questions/7125467/find-object-in-list-that-has-attribute-equal-to-some-value-that-meets-any-condi
    word = next((x for x in words if x.keyword == key), None)
    query = m[key]['query']
    page = m[key]['link'][1]
    wiki_counts = wiki.get_counts(page, word.ts_articles.getStartDate(), word.ts_articles.getEndDate(),"en")
    if wiki_counts is not None:
        #If wikipedia timeseries exists
        word.coocKeywords = query
        word.wikipediaSite = page
        word.ts_wiki = timeseries.parseWikipediaCounts(wiki_counts)

    #Same procedure for shuffled words
    word_shuffled = next((x for x in words_shuffled if x.keyword == key), None)
    query_shuffled = m[key]['query']
    page_shuffled = m[key]['link'][1]
    wiki_counts = wiki.get_counts(page, word_shuffled.ts_articles.getStartDate(), word_shuffled.ts_articles.getEndDate(),"en")
    if wiki_counts is not None:
        word_shuffled.coocKeywords = query
        word_shuffled.wikipediaSite = page
        word_shuffled.ts_wiki = timeseries.parseWikipediaCounts(wiki_counts)
    


404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found
404 Not Found


In [29]:
#Get list of all words with matched wikipedia site
words_analyze = [x for x in words if x.wikipediaSite != ""]
words_analyze_shuffled = [x for x in words_shuffled if x.wikipediaSite != ""]

corr = []
corr_shuffled = []

#Drop all timepoints which are not contained in both article and wikipedia timeseries
for w in words_analyze:
    ts_a, ts_w = timeseries.alignTimeseries(w.ts_articles,w.ts_wiki)
    if len(ts_w.getCounts())<2 or len(ts_a.getCounts())<2:
        words_analyze.remove(w)
    else:
        corr.append(statistics.getCorrelation(ts_a.getCounts(),ts_w.getCounts()))
for w in words_analyze_shuffled:
    ts_a, ts_w = timeseries.alignTimeseries(w.ts_articles,w.ts_wiki)
    if len(ts_w.getCounts())<2 or len(ts_a.getCounts())<2:
        words_analyze_shuffled.remove(w)
    else:
        corr_shuffled.append(statistics.getCorrelation(ts_a.getCounts(),ts_w.getCounts()))

2020_Democratic_Party_presidential_debates
2020_Democratic_Party_presidential_debates


In [24]:
import numpy
print("correlation")
print(numpy.mean(corr))
print(numpy.mean(corr_shuffled))

correlation
0.2321428606274169
0.28399624372371424
